In [ ]:
# making a connection to ModelOps and making sure we can see some projects

from aoa import AoaClient, ProjectApi
import pprint

client = AoaClient(
    aoa_url="https://vmo.local/core",
    auth_mode="device_code",
    auth_client_id="modelops",
    ssl_verify=False,
    auth_token_url=(
        "https://vmo.local/sso/realms/teradata/protocol/openid-connect/token"
    ),
    auth_device_auth_url=(
        "https://vmo.local/sso/realms/teradata/protocol/openid-connect/auth/device"
    ),
)

project_api = ProjectApi(aoa_client=client)

projects = list(project_api)
pprint.pprint(projects)

In [ ]:
# notice a file could be with any name, we just upload it as model.pmml
file = "./pima.pmml"
language = "PMML"
client.project_id = "23e1df4b-b630-47a1-ab80-7ad5385fcd8d"

In [ ]:
# Searching for a dataset template

from aoa import DatasetTemplateApi

dataset_template_api = DatasetTemplateApi(aoa_client=client)

dataset_template = dataset_template_api.find_by_name_like("PIMA")["_embedded"][
    "datasetTemplates"
][0]
dataset_template

In [ ]:
# Using that dataset template, let's fetch datasets ...

from aoa import DatasetApi

dataset_api = DatasetApi(aoa_client=client)
datasets = dataset_api.find_by_dataset_template_id(dataset_template["id"])["_embedded"][
    "datasets"
]
datasets

In [ ]:
# ... and take train dataset

train_dataset = [d for d in datasets if d["scope"] == "train"][0]
train_dataset

In [ ]:
# New API, allows users do fetch a default dataset connection
# (notice it only works for real users, service account probably don't have personal connections)

from aoa import UserAttributesApi

user_attributes_api = UserAttributesApi(aoa_client=client)
default_connection = user_attributes_api.get_default_connection()
default_connection
default_connection_id = default_connection["value"]["defaultDatasetConnectionId"]
default_connection_id

In [ ]:
# Let's create a BYOM model that we'll use to register model versions in

from aoa import ModelApi
import uuid

model_api = ModelApi(aoa_client=client)
model_dict = {
    "name": f"{language}_Python_{uuid.uuid4().clock_seq}",
    "description": f"{language} model defined from Python SDK",
    "language": language,
}
model_response = model_api.save(model_dict)
model = model_response["id"]

In [ ]:
# Importing that artefact, all we need is a language/format, and path to a local file

from aoa import TrainedModelArtefactsApi

trained_model_artefacts_api = TrainedModelArtefactsApi(aoa_client=client)
import_id = trained_model_artefacts_api.upload_byom_model("PMML", file)
import_id

In [ ]:
# Second step of BYOM is here. We define a structure with all necessary metadata and submit our import request

import_request = {
    "artefactImportId": import_id,
    "externalId": str(uuid.uuid4()),
    "modelMonitoring": {
        "language": language,
        "useDefaultEvaluation": True,
        "evaluationEnabled": True,
        "modelType": "CLASSIFICATION",
        "byomColumnExpression": (
            "CAST(CAST(json_report AS JSON).JSONExtractValue('$.predicted_HasDiabetes')"
            " AS INT)"
        ),
        "driftMonitoringEnabled": True,
        "datasetId": train_dataset["id"],
        "datasetConnectionId": default_connection_id,
    },
}

response = model_api.import_byom(model, import_request)

In [ ]:
response